In [1]:
import pandas as pd

In [2]:
months = {1: "01",
          2: "02",
          3: "03",
          4: "04",
          5: "05",
          6: "06",
          7: "07",
          8: "08",
          9: "09",
          10: "10",
          11: "11",
          12: "12"}

days_in_month = {1: 31,
                 2: 28,
                 3: 31,
                 4: 30,
                 5: 31,
                 6: 30,
                 7: 31,
                 8: 31,
                 9: 30,
                 10: 31,
                 11: 30,
                 12: 31}

days_to_str = {1: "01",
               2: "02",
               3: "03",
               4: "04",
               5: "05",
               6: "06",
               7: "07",
               8: "08",
               9: "09",
               10: "10",
               11: "11",
               12: "12",
               13: "13",
               14: "14",
               15: "15",
               16: "16",
               17: "17",
               18: "18",
               19: "19",
               20: "20",
               21: "21",
               22: "22",
               23: "23",
               24: "24",
               25: "25",
               26: "26",
               27: "27",
               28: "28",
               29: "29",
               30: "30",
               31: "31"}

In [3]:
### Emissions and Supply ###

# Calculates the average emission per hour on a given day
def get_es_hour_range(curr_year, curr_month, day, df, num, st_hour = 1, end_hour = 24):
    
    month = months[curr_month]
    
    curr_day_emissions = pd.read_csv("CAISO-co2-" + curr_year + month + days_to_str[day] + ".csv")
    curr_day_emissions = curr_day_emissions.fillna(4000)
  
    curr_day_supply = pd.read_csv("CAISO-supply-" + curr_year + month + days_to_str[day] + ".csv")
    curr_day_supply = curr_day_supply.fillna(0)
    
    day_supply = 0
    good_day_supply = 0
    
    day_emissions = 0
    good_day_emissions = 0

    for i in range(st_hour, end_hour + 1): # Iterates through the start of every hour - 1 to 24
        for j in range(0, 7): #iterate through each row of diff supplies
            if(curr_day_supply.iloc[j, i] > 0 ):
                good_day_supply = curr_day_supply.iloc[j, i * 11]
                day_supply += curr_day_supply.iloc[j, i * 11]
            else:
                day_supply += good_day_supply
        
        if(curr_day_emissions.iloc[0, i * 11]  > 0 ):
            good_day_emissions = curr_day_emissions.iloc[0, i * 11] 
            day_emissions += curr_day_emissions.iloc[0, i * 11] 
        else:
            day_emissions += good_day_emissions
        
        df.iloc[num[0]] = [day_supply, day_emissions, day_emissions/day_supply]   
        num[0] += 1
    
    return

# Calculates the average emission per day
def get_es_day_range(curr_year, curr_month, num_days, df, num):
    
    start = 1
    if(curr_month == 10): # Removing potentially bad data (Replace Oct 1 and 2 with 3 and 4)
        for k in range(3, 5):
            get_es_hour_range(curr_year, curr_month, k, df, num)
        start = 3

   
    for i in range(start, num_days + 1):
        get_es_hour_range(curr_year, curr_month, i, df, num) # Will just do the whole day
        
    return

# Calculates the average emission per month
def get_es_month_range(curr_year, st_month, end_month, df, num):
    
    num_months = 0
    
    for i in range(st_month, end_month + 1):
        get_es_day_range(curr_year, i, days_in_month[i], df, num) # Will just do the whole month
        num_months += 1
    return


# Calculates the annual emission from Feb 2019 to Jan 2020
def get_annual_es(df):
    
    num = [0]
    get_es_month_range("2019", 2, 12, df, num)
    get_es_month_range("2020", 1, 1, df, num)
    
    

In [4]:
df = pd.DataFrame(index=[pd.Series(range(24 * 365)).values], columns = ["Emissions/Supply (C02 kg/kwH)", "Supply (gW/hr)", "Emissions (mmT/hr)"])
get_annual_es(df)
df.to_csv('emissions-supply.csv')

df

,Emissions/Supply (C02 kg/kwH),Supply (gW/hr),Emissions (mmT/hr)
0,26618,6448,0.242242
1,46440,12739,0.274311
2,65941,19005,0.288212
3,85426,25234,0.29539
4,105417,31519,0.298994
5,126436,37838,0.299266
6,157186,45019,0.286406
7,189865,52753,0.277845
8,220006,59257,0.269343
9,249060,64711,0.259821
